## Introduction

Is is possible to predict the gender of a character in a story using the pronouns around them?

Yes

He and she are objective pronouns

Him and her are subjective pronouns

His and her are possessive adjectives

His and hers are possessive pronouns

"Her" is possessive when followed by a noun.

## Methodology

## Hypothesis



In [87]:
%pip install sparknlp
%pip install pyspark
%pip install spacy
import sparknlp
import os
import spacy
import pyspark
from pyspark.sql import SparkSession
from tqdm import tqdm
spark = sparknlp.start()

In [88]:
root = os.path.dirname(os.path.realpath('sparknlp.ipynb'))
file = open(root + '/sample_data/pg1342.txt',"r")
pride = file.read()

In [89]:
from sparknlp.pretrained import PretrainedPipeline
pipeline = PretrainedPipeline("explain_document_ml")

explain_document_ml download started this may take some time.
Approx size to download 9 MB
[OK!]


In [90]:
#result = pipeline.annotate(pride)

In [91]:
# tok_tag = [(result['token'], result['pos'])]
# zips = [list(zip(tt[0],tt[1])) for tt in tok_tag]

In [92]:
pronouns = ['he','him','his','she','her','hers']
def pronoun_check(x):
  '''
  x = list of tuples
  returns male and female pronoun count
  '''
  pro_toks = []
  male_count, female_count = 0, 0
  for i in x:
    flag = False
    last_pos = ''
    last_tok = ''
    for y in i:
      token, pos = y
      if flag:
        if pos != 'NN' and pos != 'NNS' and token == '.':
          female_count += 1
          flag = False
          continue
      token = token.lower()
      # Magic (identifies instances of 'her' that are objective)
      if token in pronouns:
        if token == 'her':
          flag = True
          last_pos = pos
          last_tok = token
        else:
            if token == 'he' or token == 'him':
              male_count += 1
            else:
              female_count += 1

  return male_count, female_count

In [93]:
# class Node:
#   def __init__(self, data):
#     # reference to next node in DLL
#     self.next = None
#     # reference to previous node in DLL
#     self.prev = None
#     self.data = data

# def insert_at_end(head, data):
#   # insert new node at the end of the list
#   new_node = Node(data)
#   if head is None:
#     return new_node

#   current = head
#   while current.next:
#     current = current.next

#   current.next = new_node
#   new_node.prev = current
#   return head

# def display(head):
#   current = head
#   while current:
#     print(current.data, end = " <-> ")
#     current = current.next
#   print(None)

In [94]:
# head = None
# head = insert_at_end(head, 1)
# head = insert_at_end(head, 2)
# head = insert_at_end(head, 3)

# print("Doubly Linked List after insertion at the end:")
# display(head)

In [95]:
pride = pride.replace('Mr.','Mr ')
pride = pride.replace('Mrs.','Mrs ')
pride = pride.replace('Ms.','Ms ')
pride = pride.replace('Dr.','Dr ')
pride = pride.replace('_',' ')
pride = pride.replace('-',' ')
pride = pride.replace('\n',' ')
pride = pride.replace('""',' ')
pride = pride.replace('  ',' ')
# pride = pride.lower() # caps help our NER
pridesplit = pride.split('.')

In [96]:
def period(sentences):
  sent_list = []
  for sent in sentences:
    sent = sent+'.'
    sent_list.append(sent)
  return sent_list

In [97]:
pridesplit = period(pridesplit)

In [98]:
# for sent in pridesplit:
#   results = []
#   result = pipeline.annotate(sent)
#   results.append(result)
# return results

In [99]:
nlp = spacy.load('en_core_web_sm')
personGenderCount = {} # person: malePronounCount, femalePronounCount
# for sent in pridesplit:
for sent in tqdm(pridesplit): # for each sentence in pride
  doc = nlp(sent) # create the spacy doc
  people = []
  for ent in doc.ents:
    if ent.label_ == 'PERSON': # get all people entities
      people.append(ent.text)
  if len(people) == 0: # if we didn't find any people
    continue #  move on to next sentence (for now)
  annoted_sent = pipeline.annotate(sent) # annotate with parts of speach
  tok_tag = [(annoted_sent['token'], annoted_sent['pos'])] # get the tokens and parts of speech
  zips = [list(zip(tt[0],tt[1])) for tt in tok_tag] # get the pronoun zips
  male_count, female_count = pronoun_check(zips) # count the pronouns
  if male_count + female_count == 0: # if we didn't find any pronouns
    continue #  move on to next sentence
  for person in people: # for each person in the sentence
    try: # if they already exist in the dict
      # add the new counts to the old
      person_male_count = personGenderCount[person][0]
      person_female_count = personGenderCount[person][1]
      personGenderCount[person.lower()] = person_male_count + male_count, person_female_count + female_count
    except KeyError: # if they don't exist in the dict
      personGenderCount[person.lower()] = male_count, female_count # add them to the dict


personGenderCount


100%|██████████| 5516/5516 [10:32<00:00,  8.72it/s]


{'comyns carr            ': (0, 1),
 'mary': (0, 4),
 'crawford': (2, 0),
 'mrs norris': (0, 1),
 'miss bates': (0, 1),
 'miss austen': (0, 1),
 'austen': (0, 1),
 'darcy': (0, 1),
 'elizabeth': (0, 1),
 'john thorpe': (0, 1),
 'swift': (0, 2),
 'smollett': (0, 1),
 'miss burney': (0, 1),
 'buff': (1, 3),
 'lydia bennet': (0, 2),
 'mrs musgrove’s': (0, 1),
 'bennet': (1, 3),
 'goldwin smith': (1, 1),
 'memling': (0, 1),
 'meissonier': (0, 1),
 'collins': (0, 1),
 'jane': (0, 1),
 'mrs bennet': (0, 1),
 'john dashwood': (1, 0),
 'lady catherine de bourgh': (0, 1),
 'mr collins': (0, 1),
 'lady catherine': (1, 3),
 'austen leigh': (0, 2),
 'bingleys': (1, 1),
 'miss de bourgh': (1, 4),
 'wickham': (0, 1),
 'don juanish': (2, 1),
 'pemberley': (2, 0),
 'lizzy': (1, 0),
 'mrs long': (0, 1),
 'michaelmas': (3, 1),
 'mr bingley': (0, 2),
 'mr bennet': (1, 2),
 'kitty': (0, 1),
 'bingley': (0, 3),
 'william': (1, 1),
 'george allen': (0, 1),
 'mrs hurst': (0, 1),
 'elizabeth bennet': (0, 1),


In [100]:
# convert personGenderCount to list
personGenderList = []
for key, value in personGenderCount.items():
  m_count, f_count = value[0], value[1]
  if (m_count == 0 and f_count == 0) or (m_count + f_count < 10):
    continue
  personGenderList.append([key, value[0], value[1]])


In [ ]:
out_list = [] # will consist of [name, prediction, actual]
for person in personGenderList:
  name, male_count, female_count = person[0], person[1], person[2]
  prediciton = 1 if male_count > female_count else 0
  actual = input('Is ' + name + ': ' + str(prediciton) + '? ')
  out_list.append([name, prediciton, actual])

In [ ]:
import pandas as pd
import datetime
df = pd.DataFrame(out_list, columns=['name', 'prediction', 'actual'])
df = df.astype({'actual': int})
df.to_csv('output_' + datetime.datetime.now().strftime("%Y-%m-%d") + '.csv', index=False)
df = df[df['actual'] != -1] # drop locations, after filesave (for scoring)


In [ ]:
df

In [ ]:
# calculate accuracy, when predicted == actual that's good
accuracy = (df['prediction'] == df['actual']).sum() / len(df)
print(f'Accuracy: {round(accuracy*100,2)}%')

In [ ]:
# female accuracy
female_df = df[df['actual'] == 0]
female_accuracy = (female_df['prediction'] == female_df['actual']).sum() / len(female_df)
print(f'Female Accuracy: {round(female_accuracy*100,2)}%')
# male accuracy
male_df = df[df['actual'] == 1]
male_accuracy = (male_df['prediction'] == male_df['actual']).sum() / len(male_df)
print(f'Male Accuracy: {round(male_accuracy*100,2)}%')
